In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from folium.plugins import Search


In [2]:
df = pd.read_csv("data/europisco.csv")

In [3]:
df.drop(columns=["Email Address", "Cuál es tu correo? (opcional)"], inplace=True)

In [4]:
df.rename(
    columns={
        "Timestamp": "timestamp",
        "Cúal es la dirección exacta (o más aproximada) dónde viste el pisco?": "address_raw",
        "En qué país?": "country",
        "Qué era específicamente?": "pisco_type",
        "Cúanto costaba aproximadamente (en euros)?": "price",
        "Cuál es tu nombre? (opcional)": "name",
        "Probable address": "address_clean"
    },
    inplace=True
)

In [5]:
df.dropna(subset=['address_clean'], inplace=True)

In [6]:
df.reset_index(inplace=True, drop=True)

In [7]:
df

,timestamp,address_raw,country,pisco_type,price,name,address_clean
0,6/8/2024 22:34:22,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",España,Botella,17.0,Joaquín Arretx,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar..."
1,6/14/2024 17:40:03,"Ham 31, 9000 Gent",Bélgica,Botella,25.0,Sebastian,"Ham 31, 9000 Gent"
2,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Prenzlauer Promenade 192, 13189 Berlin, Germany"
3,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Kantstraße 42, 10625 Berlin, Germany"
4,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Berliner Str. 78, 14169 Berlin, Germany"
...,...,...,...,...,...,...,...
111,6/27/2024 22:39:08,"8 Rue de Birague, 75004 Paris",Francia,"Piscola, Pisco Sour",9.0,NaN,"8 Rue de Birague, 75004 Paris, France"
112,6/27/2024 23:47:11,"Østerbrogade 93, 2100 København Ø",Dinamarca,Botella,15.0,Matilde,"Østerbrogade 93, 2100 København, Denmark"
113,6/28/2024 9:59:49,"C. de Preciados, 3, Centro, 28013 Madrid (Cort...",España,Botella,22.0,Camilo,"C. de Preciados, 3, Centro, 28013 Madrid"
114,6/28/2024 14:13:00,"Atelierstraße 5-7, 81671 München, Alemania",Alemania,Botella,17.5,Rafa,"Atelierstraße 5-7, 81671 München, Germany"


In [8]:
geolocator = Nominatim(user_agent="BuscandoElPisco")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Create new columns for latitude and longitude
df['location'] = df['address_clean'].apply(geocode)
df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('R. de Santa Justa 18, 1100-485 Lisboa, Portugal',), **{}).
Traceback (most recent call last):
  File "/home/jarretx/europisco/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/home/jarretx/europisco/venv/lib/python3.10/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes,

In [10]:
df.query("location.isnull()")

,timestamp,address_raw,country,pisco_type,price,name,address_clean,location,latitude,longitude
6,6/14/2024 17:58:38,Bar Juana Brava en Salamanca,España,Piscola,6.0,Fabián,"Calle de Van Dyck, 13, 37005 Salamanca",None,NaN,NaN
18,6/15/2024 14:18:11,"Wine House, Rda. de Sant Antoni, 15, L'Eixampl...",España,Botella,23.0,NaN,"Wine House, Rda. de Sant Antoni, 15, L'Eixampl...",None,NaN,NaN
21,6/15/2024 15:49:08,"Bar L'Amistat, RAMON Y CAJAL 58, TORRIJOS 13, ...",España,Piscola,8.0,Joaquín Arretx,"RAMON Y CAJAL 58, TORRIJOS 13, 08012 Barcelona",None,NaN,NaN
23,6/16/2024 18:26:28,"Gran Vía de Don Diego López de Haro, 9, Abando...",España,Botella,13.0,Nico,"Gran Vía de Don Diego López de Haro, 9, Abando...",None,NaN,NaN
30,6/16/2024 19:44:25,El corte inglés Valencia Centro,España,Botella,35.0,"Eduardo ""Piscola"" Segovia","Calle del Pintor Sorolla, 26, 46002, Ciutat Ve...",None,NaN,NaN
32,6/16/2024 20:23:57,Barrio de gracia Barcelona,España,"Piscola, Pisco Sour",15.0,Bar kiltro,"Carrer del Montseny, 13, Local 2, Gracia, 0801...",None,NaN,NaN
45,6/16/2024 22:02:36,En la Garrafeira Nacional en Lisboa www.garraf...,Portugal,Botella,27.0,Erika,"R. de Santa Justa 18, 1100-485 Lisboa, Portugal",None,NaN,NaN
47,6/16/2024 22:18:21,"45 Hogan Pl, Dublin 2, D02 RF34",Irlanda,Pisco Sour,12.0,Dani,"45 Hogan Pl, Dublin 2, D02 RF34, Ireland",None,NaN,NaN
48,6/16/2024 22:19:24,"Charlotte Quay Dock, Millennium Tower, Ground ...",Irlanda,Pisco Sour,13.0,Dani,"Charlotte Quay Dock, Millennium Tower, Ground ...",None,NaN,NaN
49,6/16/2024 23:06:02,"Bar L’Amistat: RAMON Y CAJAL 58, TORRIJOS 13, ...",España,Piscola,8.0,Nico,"Bar L’Amistat: RAMON Y CAJAL 58, TORRIJOS 13, ...",None,NaN,NaN


In [19]:
# df.loc[0, "latitude"] = 41.38006562275075
# df.loc[0, "longitude"] = 2.163010839291413
df.loc[6, "latitude"] = 40.972546300325746
df.loc[6, "longitude"] = -5.658419387328101
df.loc[18, "latitude"] = 41.38006562275075
df.loc[18, "longitude"] = 2.163010839291413
df.loc[21, "latitude"] = 41.40293279969603
df.loc[21, "longitude"] = 2.159256096961093
df.loc[23, "latitude"] = 43.26163372566617
df.loc[23, "longitude"] = -2.929120933831521
df.loc[30, "latitude"] = 39.47078392527955
df.loc[30, "longitude"] = -0.37138733009770314
df.loc[32, "latitude"] = 41.40191119891952
df.loc[32, "longitude"] = 2.154507805637604
df.loc[45, "latitude"] = 38.7127794867824
df.loc[45, "longitude"] = -9.137327763919329
df.loc[47, "latitude"] = 53.34083826660188
df.loc[47, "longitude"] = -6.24482527367974
df.loc[48, "latitude"] = 53.342904383050765
df.loc[48, "longitude"] = -6.236903285325631
df.loc[49, "latitude"] = 41.40290997130236
df.loc[49, "longitude"] = 2.1592672681276954
df.loc[51, "latitude"] = 51.90172595208701
df.loc[51, "longitude"] = -8.470310360254798
df.loc[52, "latitude"] = 38.709764127838696
df.loc[52, "longitude"] = -9.151407431973473
df.loc[59, "latitude"] = 46.22150670601297
df.loc[59, "longitude"] = 6.0956530670457125
df.loc[69, "latitude"] = 40.4292948540896
df.loc[69, "longitude"] = -3.673644174454475
df.loc[73, "latitude"] = 38.964916663935924
df.loc[73, "longitude"] = -9.417234623696746
df.loc[86, "latitude"] = 39.61589090757615
df.loc[86, "longitude"] = 2.705252368571537
df.loc[88, "latitude"] = 38.909667696473214
df.loc[88, "longitude"] = 1.4355332133343959
df.loc[113, "latitude"] = 40.41763629833717
df.loc[113, "longitude"] = -3.704811116568445

In [13]:
df.head(5)

,timestamp,address_raw,country,pisco_type,price,name,address_clean,location,latitude,longitude
0,6/8/2024 22:34:22,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",España,Botella,17.0,Joaquín Arretx,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...","(Farmàcia Ventura Cebrià,Enric, 15, Ronda de S...",41.379950,2.163068
1,6/14/2024 17:40:03,"Ham 31, 9000 Gent",Bélgica,Botella,25.0,Sebastian,"Ham 31, 9000 Gent","(31, Huidevetterskaai, Waterwijk, Gent, Oost-V...",51.060588,3.728483
2,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Prenzlauer Promenade 192, 13189 Berlin, Germany","(Mitte Meer, 192, Prenzlauer Promenade, Prenzl...",52.551766,13.429981
3,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Kantstraße 42, 10625 Berlin, Germany","(Mitte Meer, 42, Kantstraße, Charlottenburg, C...",52.506538,13.312701
4,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Berliner Str. 78, 14169 Berlin, Germany","(Mitte Meer, 78, Berliner Straße, Zehlendorf, ...",52.439565,13.276804


In [20]:
df.to_csv("./processed/europisco_location.csv", index=False)

In [21]:
# Create a map centered around Europe
m = folium.Map(location=[54.5260, 15.2551], zoom_start=4)

# Define all possible pisco types
pisco_types = ["Botella", "Piscola", "Pisco Sour"]

# Create a feature group for each pisco type and combination
feature_groups = {ptype: folium.FeatureGroup(name=ptype).add_to(m) for ptype in pisco_types}

# Function to add markers to the map
def add_marker(row):
    popup_text = f"Tipo: {row['pisco_type']}<br>Precio: {row['price']}<br>Subido por: {row['name']}<br>Dirección: {row['address_clean']}"
    pisco_types_in_row = row['pisco_type'].split(", ")
    for pisco_type in pisco_types_in_row:
        if pisco_type in feature_groups:
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup_text,
                tooltip=row['pisco_type']
            ).add_to(feature_groups[pisco_type])

# Apply the function to each row in the dataframe
df.apply(add_marker, axis=1)

# Add LayerControl to the map
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('out/pisco_map.html')

# Stats

In [17]:
df.query("pisco_type.str.contains('botella', case=False)").sort_values("price")

,timestamp,address_raw,country,pisco_type,price,name,address_clean,location,latitude,longitude
19,6/15/2024 15:09:22,El Corte Inglés supermercado nuevos ministerio...,España,Botella,1.0,Paulina,"C. de Raimundo Fernández Villaverde, 65, Tetuá...","(EY, 65, Calle de Raimundo Fernández Villaverd...",40.446904,-3.694417
79,6/25/2024 10:06:18,"Carrer de l'Actriu Encarna Máñez, 3, Algirós, ...",España,"Botella, Piscola, Pisco Sour",6.0,Eduardo,"Carrer de l'Actriu Encarna Máñez, 3, Algirós, ...","(Carrer de l'Actriu Encarna Máñez, Ciutat Jard...",39.472537,-0.346136
25,6/16/2024 19:13:11,Via Luigi Capana 4,Italia,"Botella, Piscola",8.0,Paolo,"Via Luigi Capuana, 4, 00137 Roma RM, Italy","(Via Luigi Capuana, Monte Sacro Alto, Municipi...",41.948771,12.547797
73,6/25/2024 3:57:35,Ericeira-Portugal (Cheers Bar),Portugal,"Botella, Piscola, Pisco Sour",9.0,Victor,"R. dos Ferreiros 5, 2655-279 Ericeira, Portugal",None,38.964917,-9.417235
68,6/23/2024 21:28:32,"Montseny 13, local 2. Barcelona 08012",España,"Botella, Piscola, Pisco Sour",10.0,Franvis,"C. del Montseny, 13, Gràcia, 08012 Barcelona","(13, Carrer del Montseny, la Vila de Gràcia, G...",41.401810,2.154444
...,...,...,...,...,...,...,...,...,...,...
30,6/16/2024 19:44:25,El corte inglés Valencia Centro,España,Botella,35.0,"Eduardo ""Piscola"" Segovia","Calle del Pintor Sorolla, 26, 46002, Ciutat Ve...",None,39.470784,-0.371387
51,6/17/2024 0:59:11,"21 Coburg St, Victorian Quarter, Cork, T23 CR23",Irlanda,Botella,40.0,Marcela Bravo,"21 Coburg St, Victorian Quarter, Cork, T23 CR2...",None,51.901726,-8.470310
9,6/14/2024 20:34:17,"Industriepark-Noord 35, 9100 Sint-Niklaas",Bélgica,Botella,50.0,Milena,"Industriepark-Noord 35, 9100 Sint-Niklaas","(De Vidts, 35, Industriepark Noord, Hertje, Si...",51.151284,4.160866
89,6/25/2024 21:14:34,"Route François-Arsent 1, 1700 fribourg",Suiza,Botella,60.0,Alvaro,"Rte François-Arsent 1, 1700 Fribourg, Switzerland","(Route François-Arsent, Auge, Schönberg/Schoen...",46.806564,7.169214
